# ch10 时间序列
时间序列的数据意义：
+ 时间戳（timestamp），特定的时刻；
+ 固定时期（period），如2017年1月；
+ 时间间隔（interval），由起始和结束时间戳表示；
+ 实验或过程时间，每个时间点都是相对于特定起始时间的一个度量。

## 日期和时间数据类型及工具

Python标准库包含用于日期（date）和时间（time）数据的数据类型,还有日历方面的功能；主要会用到datetime、time、calendar模块。

In [1]:
from datetime import datetime;
now = datetime.now()
now

datetime.datetime(2017, 12, 7, 10, 16, 40, 512000)

In [2]:
now.year,now.month,now.day

(2017, 12, 7)

datetime以毫秒形式储存日期和时间。datetime.timedelta表示两个datetime对象之间的时间差

In [3]:
delta = datetime(2017,12,1)-datetime(1992,2,14,8,15)
delta

datetime.timedelta(9421, 56700)

In [4]:
delta.days

9421

In [5]:
delta.seconds

56700

In [6]:
from datetime import timedelta;
start = datetime(2011,1,7)
start + timedelta(12)

datetime.datetime(2011, 1, 19, 0, 0)

In [7]:
start -2 * timedelta(12)

datetime.datetime(2010, 12, 14, 0, 0)

表 10-1 datetime模块中的数据类型

| 类型        | 说明                   |
|:------------- |:-------------|
| date    | 以公历形式存储日历日期（年、月、日）   |
| time     | 将时间存储为时、分、秒、毫秒   |
| datetime    | 存储日期和时间 |
| timedelta   | 表示两个datetime值之间的差（日、秒、毫秒） |

### 字符串和datetime的相互转换

利用str或strftime方法（传入一个格式化字符串），datetime对象和pandas的Timestamp对象可以被格式化为字符串

In [8]:
stamp = datetime(2011,1,3)
str(stamp)

'2011-01-03 00:00:00'

In [9]:
stamp.strftime('%Y-%m-%d')

'2011-01-03'

datetime.strptime是通过已知格式进行日期解析的最佳方式。

In [10]:
value = '2011-01-03'
datetime.strptime(value,'%Y-%m-%d')

datetime.datetime(2011, 1, 3, 0, 0)

In [11]:
datestrs = ['7/6/2011','8/6/2011']
datestrs

['7/6/2011', '8/6/2011']

In [12]:
[datetime.strptime(x,'%m/%d/%Y') for x in datestrs]

[datetime.datetime(2011, 7, 6, 0, 0), datetime.datetime(2011, 8, 6, 0, 0)]

但每次编写格式定义很麻烦，对于一些常用的日期格式可以用dateutil这个第三方包中的parser.parse方法：

In [13]:
from dateutil.parser import parse
parse('2011-01-03')

datetime.datetime(2011, 1, 3, 0, 0)

dateutil可以解析几乎所有人类能够理解的日期表示形式（中文不行）：

In [14]:
parse('Jan 31,1997 10:45 PM')

datetime.datetime(2017, 1, 31, 22, 45)

In [15]:
parse('6/12/2011',dayfirst=True)

datetime.datetime(2011, 12, 6, 0, 0)

In [16]:
datestrs

['7/6/2011', '8/6/2011']

pandas通常是用于处理成组日期的，不管这些日期时DateFrame的轴索引还是列。to_datetime方法可以解析多种不同的日期表现形式。

In [17]:
import pandas as pd
pd.to_datetime(datestrs)

DatetimeIndex(['2011-07-06', '2011-08-06'], dtype='datetime64[ns]', freq=None)

还可以处理缺失值：

In [18]:
idx = pd.to_datetime(datestrs + [None])
idx

DatetimeIndex(['2011-07-06', '2011-08-06', 'NaT'], dtype='datetime64[ns]', freq=None)

In [19]:
idx[2]

NaT

In [20]:
pd.isnull(idx)

array([False, False,  True], dtype=bool)

## 时间序列基础

pandas最基本的时间序列类型就是以时间戳（通常以Python字符串或datetime对象表示）为索引的Series：

In [21]:
from datetime import datetime
dates = [datetime(2011,1,2),datetime(2011,1,5),datetime(2011,1,7),
        datetime(2011,1,8),datetime(2011,1,10),datetime(2011,1,12)]

In [22]:
import numpy as np
ts = pd.Series(np.random.randn(6),index=dates)
ts

2011-01-02   -1.610606
2011-01-05    0.179673
2011-01-07   -0.344007
2011-01-08    0.150177
2011-01-10    0.189054
2011-01-12   -0.320047
dtype: float64

这些datetime对象实际上是被放在一个DatetimeIndex中的。现在变量ts就成为一个TimeSeries了：

In [23]:
type(ts)

pandas.core.series.Series

In [24]:
ts.index

DatetimeIndex(['2011-01-02', '2011-01-05', '2011-01-07', '2011-01-08',
               '2011-01-10', '2011-01-12'],
              dtype='datetime64[ns]', freq=None)

In [25]:
ts + ts[::2]

2011-01-02   -3.221212
2011-01-05         NaN
2011-01-07   -0.688013
2011-01-08         NaN
2011-01-10    0.378108
2011-01-12         NaN
dtype: float64

pandas用NumPy的datetime64数据类型以纳秒形式存储时间戳：

In [26]:
ts.index.dtype

dtype('<M8[ns]')

In [27]:
stamp = ts.index[0]
stamp

Timestamp('2011-01-02 00:00:00')

### 索引、选取、子集构造

由于TimeSeries是Series的一个子类，所以在索引以及数据选取方面它们的行为是一样的：

In [28]:
stamp = ts.index[2]
ts[stamp]

-0.34400658304597254

In [29]:
ts['1/10/2011']

0.18905405301328962

In [30]:
ts['20110110']

0.18905405301328962

对于较长的时间序列，只需传入“年”或“年月”即可轻松选取数据的切片：

In [31]:
longer_ts = pd.Series(np.random.randn(1000),
                   index=pd.date_range('1/1/2000',periods=1000))
longer_ts

2000-01-01    1.491489
2000-01-02   -0.263160
2000-01-03    0.612917
2000-01-04   -0.855229
2000-01-05    0.670707
2000-01-06   -0.533937
2000-01-07    1.326303
2000-01-08    0.665961
2000-01-09    0.017224
2000-01-10    0.966279
2000-01-11    0.411103
2000-01-12   -0.410998
2000-01-13   -0.823098
2000-01-14   -1.449170
2000-01-15    2.233322
2000-01-16   -0.391430
2000-01-17   -1.539917
2000-01-18   -1.013574
2000-01-19    0.054730
2000-01-20    1.669596
2000-01-21   -1.133906
2000-01-22    1.550096
2000-01-23    0.982132
2000-01-24    1.164397
2000-01-25   -1.381198
2000-01-26   -0.180285
2000-01-27   -1.096084
2000-01-28    0.690112
2000-01-29    0.669354
2000-01-30   -0.314902
                ...   
2002-08-28    0.292722
2002-08-29   -0.327709
2002-08-30   -1.209994
2002-08-31   -2.111299
2002-09-01    1.276951
2002-09-02   -0.286815
2002-09-03   -1.748960
2002-09-04   -0.701125
2002-09-05   -1.184528
2002-09-06   -0.765805
2002-09-07    0.549870
2002-09-08   -1.126778
2002-09-09 

In [32]:
longer_ts['2001']

2001-01-01   -2.410923
2001-01-02   -0.150866
2001-01-03    0.384696
2001-01-04    0.511499
2001-01-05    0.940262
2001-01-06    0.681104
2001-01-07   -1.315621
2001-01-08   -0.460693
2001-01-09    1.108769
2001-01-10   -1.173708
2001-01-11   -0.395329
2001-01-12   -0.043807
2001-01-13    1.859544
2001-01-14   -0.243564
2001-01-15   -1.357741
2001-01-16    0.923180
2001-01-17    0.329499
2001-01-18    0.854616
2001-01-19   -0.001281
2001-01-20   -0.264639
2001-01-21   -2.115772
2001-01-22    0.285079
2001-01-23   -1.194870
2001-01-24    0.075325
2001-01-25    1.023701
2001-01-26    0.713623
2001-01-27   -1.164796
2001-01-28   -0.718235
2001-01-29   -0.409325
2001-01-30    0.515600
                ...   
2001-12-02    0.784725
2001-12-03    0.756419
2001-12-04   -0.333629
2001-12-05    1.506195
2001-12-06    0.243603
2001-12-07    1.835346
2001-12-08   -1.202496
2001-12-09   -0.120856
2001-12-10    0.281229
2001-12-11    2.018776
2001-12-12    0.198460
2001-12-13    1.561357
2001-12-14 

In [33]:
longer_ts['2001-05']

2001-05-01    0.090918
2001-05-02   -0.706175
2001-05-03    0.091847
2001-05-04    1.710950
2001-05-05   -1.250379
2001-05-06   -0.339324
2001-05-07    0.608858
2001-05-08   -0.140174
2001-05-09    0.941644
2001-05-10   -0.313011
2001-05-11   -0.314101
2001-05-12    0.293547
2001-05-13    0.022432
2001-05-14   -1.201698
2001-05-15    0.532952
2001-05-16    1.648295
2001-05-17   -1.005527
2001-05-18   -0.079510
2001-05-19    0.564685
2001-05-20    1.165465
2001-05-21    0.214370
2001-05-22    0.150339
2001-05-23    0.948473
2001-05-24    1.046552
2001-05-25   -0.801782
2001-05-26    0.265802
2001-05-27   -0.440364
2001-05-28    0.668902
2001-05-29   -0.243596
2001-05-30   -0.254731
2001-05-31   -0.625122
Freq: D, dtype: float64

通过日期进行切片的方式只对规则Series有效：

In [34]:
ts[datetime(2011,1,7):]

2011-01-07   -0.344007
2011-01-08    0.150177
2011-01-10    0.189054
2011-01-12   -0.320047
dtype: float64

In [35]:
ts

2011-01-02   -1.610606
2011-01-05    0.179673
2011-01-07   -0.344007
2011-01-08    0.150177
2011-01-10    0.189054
2011-01-12   -0.320047
dtype: float64

In [36]:
ts['1/6/2011':'1/11/2011']

2011-01-07   -0.344007
2011-01-08    0.150177
2011-01-10    0.189054
dtype: float64

跟之前一样，这里可以传入字符串日期、datetime或Timestamp。注意，这样切片所产生的源时间序列的视图，跟NumPy数组的切片运算是一样的。此外，还有一个等价的实例方法也可以截取两个日期之间的TimeSeries：

In [37]:
ts.truncate(after='1/9/2011')

2011-01-02   -1.610606
2011-01-05    0.179673
2011-01-07   -0.344007
2011-01-08    0.150177
dtype: float64

上面这些操作对DateFrame也有效。例如，对DataFrame的行进行索引：

In [38]:
dates = pd.date_range('1/1/2000',periods=100,freq='W-WED')
long_df = pd.DataFrame(np.random.randn(100,4),
                  index=dates,
                  columns=['Colorado','Texas','New York','Ohio'])

In [39]:
long_df.loc['5-2001']

,Colorado,Texas,New York,Ohio
2001-05-02,2.225538,-1.463777,-0.522076,-0.193462
2001-05-09,-2.273152,0.727469,-0.443727,-0.624931
2001-05-16,1.271429,-0.800354,-0.195998,-1.058635
2001-05-23,0.278766,0.951398,-1.856221,-1.193557
2001-05-30,-0.685990,1.264896,-0.399505,0.986024


### 带有重复索引的时间序列

In [40]:
dates = pd.DatetimeIndex(['1/1/2000','1/2/2000','1/2/2000','1/2/2000',
                         '1/3/2000'])
dup_ts = pd.Series(np.arange(5),index=dates)
dup_ts

2000-01-01    0
2000-01-02    1
2000-01-02    2
2000-01-02    3
2000-01-03    4
dtype: int32

In [41]:
dup_ts.index.is_unique

False

In [42]:
dup_ts['1/3/2000']#不重复

4

In [43]:
dup_ts['1/2/2000']#重复

2000-01-02    1
2000-01-02    2
2000-01-02    3
dtype: int32

对非唯一时间戳的数据进行聚合：

In [44]:
grouped = dup_ts.groupby(level=0)

In [45]:
grouped.mean()

2000-01-01    0
2000-01-02    2
2000-01-03    4
dtype: int32

In [46]:
grouped.count()

2000-01-01    1
2000-01-02    3
2000-01-03    1
dtype: int64

## 日期的范围、频率以及移动

pandas中的时间序列一般被认为是不规则的，没有固定的频率。但分析中常常需要以某种相对固定的频率进行分析，比如每日、每月、每15分钟等；pandas有一整套标准时间序列频率以及用于重采样、频率推断、生成固定频率日期范围的工具。

In [47]:
ts

2011-01-02   -1.610606
2011-01-05    0.179673
2011-01-07   -0.344007
2011-01-08    0.150177
2011-01-10    0.189054
2011-01-12   -0.320047
dtype: float64

In [48]:
ts.resample('D')

DatetimeIndexResampler [freq=<Day>, axis=0, closed=left, label=left, convention=start, base=0]

### 生成日期范围

In [49]:
index = pd.date_range('4/1/2012','6/1/2012')
index

DatetimeIndex(['2012-04-01', '2012-04-02', '2012-04-03', '2012-04-04',
               '2012-04-05', '2012-04-06', '2012-04-07', '2012-04-08',
               '2012-04-09', '2012-04-10', '2012-04-11', '2012-04-12',
               '2012-04-13', '2012-04-14', '2012-04-15', '2012-04-16',
               '2012-04-17', '2012-04-18', '2012-04-19', '2012-04-20',
               '2012-04-21', '2012-04-22', '2012-04-23', '2012-04-24',
               '2012-04-25', '2012-04-26', '2012-04-27', '2012-04-28',
               '2012-04-29', '2012-04-30', '2012-05-01', '2012-05-02',
               '2012-05-03', '2012-05-04', '2012-05-05', '2012-05-06',
               '2012-05-07', '2012-05-08', '2012-05-09', '2012-05-10',
               '2012-05-11', '2012-05-12', '2012-05-13', '2012-05-14',
               '2012-05-15', '2012-05-16', '2012-05-17', '2012-05-18',
               '2012-05-19', '2012-05-20', '2012-05-21', '2012-05-22',
               '2012-05-23', '2012-05-24', '2012-05-25', '2012-05-26',
      

In [50]:
pd.date_range(start='4/1/2012',periods=20)

DatetimeIndex(['2012-04-01', '2012-04-02', '2012-04-03', '2012-04-04',
               '2012-04-05', '2012-04-06', '2012-04-07', '2012-04-08',
               '2012-04-09', '2012-04-10', '2012-04-11', '2012-04-12',
               '2012-04-13', '2012-04-14', '2012-04-15', '2012-04-16',
               '2012-04-17', '2012-04-18', '2012-04-19', '2012-04-20'],
              dtype='datetime64[ns]', freq='D')

In [51]:
pd.date_range(end='1/6/2012',periods=20)

DatetimeIndex(['2011-12-18', '2011-12-19', '2011-12-20', '2011-12-21',
               '2011-12-22', '2011-12-23', '2011-12-24', '2011-12-25',
               '2011-12-26', '2011-12-27', '2011-12-28', '2011-12-29',
               '2011-12-30', '2011-12-31', '2012-01-01', '2012-01-02',
               '2012-01-03', '2012-01-04', '2012-01-05', '2012-01-06'],
              dtype='datetime64[ns]', freq='D')

起始和结束日期定义了日期索引的严格边界。例如，若想要生成一个由每月最后一个工作日组成的日期索引，可以传入“BM”频率：

In [52]:
pd.date_range('1/1/2017','12/1/2017',freq='BM')

DatetimeIndex(['2017-01-31', '2017-02-28', '2017-03-31', '2017-04-28',
               '2017-05-31', '2017-06-30', '2017-07-31', '2017-08-31',
               '2017-09-29', '2017-10-31', '2017-11-30'],
              dtype='datetime64[ns]', freq='BM')

date_range默认会保留起始和结束时间戳的信息：

In [53]:
pd.date_range('5/2/2012 12:56:31',periods=5)

DatetimeIndex(['2012-05-02 12:56:31', '2012-05-03 12:56:31',
               '2012-05-04 12:56:31', '2012-05-05 12:56:31',
               '2012-05-06 12:56:31'],
              dtype='datetime64[ns]', freq='D')

但你希望产生一组被规范化到午夜的时间戳：

In [54]:
pd.date_range('5/2/2012 12:56:31',periods=5,normalize=True)

DatetimeIndex(['2012-05-02', '2012-05-03', '2012-05-04', '2012-05-05',
               '2012-05-06'],
              dtype='datetime64[ns]', freq='D')

### 频率和日期偏移量

In [55]:
from pandas.tseries.offsets import Hour,Minute
hour = Hour()
hour

<Hour>

In [56]:
four_hours = Hour(4)
four_hours

<4 * Hours>

一般来说，无需显式创建这样的对象，只需使用诸如“H”或“4H”这样的字符串别名即可。在基础频率前面放上一个整数即可创建倍数：

In [57]:
pd.date_range('1/1/2017','1/2/2000 23:59',freq='4h')

DatetimeIndex([], dtype='datetime64[ns]', freq='4H')

大部分偏移量都可以通过加法进行连接：

In [58]:
Hour(2) + Minute(30)

<150 * Minutes>

也可以传入频率字符串（如“2h30min")

In [59]:
pd.date_range('1/1/2017',periods=10,freq='1h30min')

DatetimeIndex(['2017-01-01 00:00:00', '2017-01-01 01:30:00',
               '2017-01-01 03:00:00', '2017-01-01 04:30:00',
               '2017-01-01 06:00:00', '2017-01-01 07:30:00',
               '2017-01-01 09:00:00', '2017-01-01 10:30:00',
               '2017-01-01 12:00:00', '2017-01-01 13:30:00'],
              dtype='datetime64[ns]', freq='90T')

### WOM日期

WOM（Week Of Month)，获得诸如“每月第三个星期五”之类的日期：

In [60]:
rng = pd.date_range('1/1/2012','9/1/2012',freq='WOM-3FRI')
list(rng)

[Timestamp('2012-01-20 00:00:00', freq='WOM-3FRI'),
 Timestamp('2012-02-17 00:00:00', freq='WOM-3FRI'),
 Timestamp('2012-03-16 00:00:00', freq='WOM-3FRI'),
 Timestamp('2012-04-20 00:00:00', freq='WOM-3FRI'),
 Timestamp('2012-05-18 00:00:00', freq='WOM-3FRI'),
 Timestamp('2012-06-15 00:00:00', freq='WOM-3FRI'),
 Timestamp('2012-07-20 00:00:00', freq='WOM-3FRI'),
 Timestamp('2012-08-17 00:00:00', freq='WOM-3FRI')]

美国的股票期权交易人会意识到这些日子就是标准的月度到期日。

### 移动（超前和滞后）数据
移动（shifting）指的是沿着时间轴将数据前移或后移。Series和DataFrame都有一个shift方法用于执行单纯的前移或后移操作，保持索引不变：

In [61]:
ts = pd.Series(np.random.randn(4),
              index=pd.date_range('1/1/2017',periods=4,freq='M'))
ts

2017-01-31   -1.438781
2017-02-28   -0.380632
2017-03-31   -0.337799
2017-04-30    0.716410
Freq: M, dtype: float64

In [62]:
ts.shift(1)

2017-01-31         NaN
2017-02-28   -1.438781
2017-03-31   -0.380632
2017-04-30   -0.337799
Freq: M, dtype: float64

In [63]:
ts.shift(-2)

2017-01-31   -0.337799
2017-02-28    0.716410
2017-03-31         NaN
2017-04-30         NaN
Freq: M, dtype: float64

shift通常用于计算一个时间序列（如DataFrame的列）中的百分比变化。可以这样表达：

In [64]:
ts/ts.shift(1)-1

2017-01-31         NaN
2017-02-28   -0.735449
2017-03-31   -0.112531
2017-04-30   -3.120818
Freq: M, dtype: float64

单纯的移位操作不会修改索引，所以部分数据会被丢弃。如果频率已知，则可以将其传给shift以便实现对时间戳进行位移而不是对数据进行简单位移：

In [67]:
ts.shift(2,freq='M')

2017-03-31   -1.438781
2017-04-30   -0.380632
2017-05-31   -0.337799
2017-06-30    0.716410
Freq: M, dtype: float64

In [68]:
ts.shift(1,freq='D')

2017-02-01   -1.438781
2017-03-01   -0.380632
2017-04-01   -0.337799
2017-05-01    0.716410
Freq: MS, dtype: float64

In [69]:
ts.shift(1,freq='90T')

2017-01-31 01:30:00   -1.438781
2017-02-28 01:30:00   -0.380632
2017-03-31 01:30:00   -0.337799
2017-04-30 01:30:00    0.716410
Freq: M, dtype: float64

### 通过偏移量对日期进行位移
pandas的日期偏移量还可以用在datetime或Timestamp对象上：

In [72]:
from pandas.tseries.offsets import Day,MonthEnd
now = datetime(2011,11,17)
now + 3 * Day()

Timestamp('2011-11-20 00:00:00')

如果加的是锚点偏移量（比如MonthEnd），第一次增量会将原日期向前滚动到符合频率规则的下一个日期：

In [73]:
now + MonthEnd()

Timestamp('2011-11-30 00:00:00')

In [74]:
now + MonthEnd(2)

Timestamp('2011-12-31 00:00:00')

通过锚点偏移量的rollforward和rollback方法，可显示地将日期向前或向后“滚动”：

In [75]:
offset = MonthEnd()
offset

<MonthEnd>

In [76]:
offset.rollforward(now)

Timestamp('2011-11-30 00:00:00')

In [77]:
offset.rollback(now)

Timestamp('2011-10-31 00:00:00')

日期偏移量还有个巧妙的用法，即结合groupby使用这两个“滚动”方法：

In [82]:
ts = pd.Series(np.random.randn(20),
           index=pd.date_range('1/15/2000',periods=20,freq='4d'))
ts

2000-01-15    0.609891
2000-01-19   -1.917458
2000-01-23    1.165923
2000-01-27   -0.626803
2000-01-31   -1.135762
2000-02-04   -0.256763
2000-02-08   -1.623899
2000-02-12    0.493804
2000-02-16   -0.167435
2000-02-20    0.623102
2000-02-24    0.889468
2000-02-28   -0.172258
2000-03-03    0.665390
2000-03-07   -0.219608
2000-03-11    0.881369
2000-03-15   -0.094255
2000-03-19   -0.477023
2000-03-23    0.842384
2000-03-27   -1.255157
2000-03-31   -0.070248
Freq: 4D, dtype: float64

In [83]:
ts.groupby(offset.rollforward).mean()

2000-01-31   -0.380842
2000-02-29   -0.030569
2000-03-31    0.034106
dtype: float64

当然，更简单、快速实现该功能的办法是使用resample：

In [85]:
ts.resample('M').mean()

2000-01-31   -0.380842
2000-02-29   -0.030569
2000-03-31    0.034106
Freq: M, dtype: float64

## 时区处理
在Python中，时区信息来自第三方库pytz，它使Python可以使用Olson数据库（汇编了世界时区信息）。由于pandas包装了pytz的功能，因此只需记得时区名称即可，时区名称可以在文档中找到，也可以通过交互的方式查看：

In [87]:
import pytz
pytz.common_timezones[-5:]

['US/Eastern', 'US/Hawaii', 'US/Mountain', 'US/Pacific', 'UTC']

要从pytz中获取时区对象，使用pytz.timezone即可：

In [88]:
tz = pytz.timezone('US/Eastern')
tz

<DstTzInfo 'US/Eastern' LMT-1 day, 19:04:00 STD>

pandas中的方法既可以接受时区名也可以接受这种对象，建议只用时区名。
### 本地化和转换
默认情况下，pandas中的时间序列是单纯的（naive）时区：

In [89]:
rng = pd.date_range('3/9/2012 9:30',periods=6,freq='D')
ts = pd.Series(np.random.randn(len(rng)),index=rng)
ts

2012-03-09 09:30:00   -0.167662
2012-03-10 09:30:00    1.122850
2012-03-11 09:30:00   -1.029191
2012-03-12 09:30:00   -1.002843
2012-03-13 09:30:00   -1.324037
2012-03-14 09:30:00   -0.217383
Freq: D, dtype: float64

其索引的tz字段为None：

In [91]:
print(ts.index.tz)

None


在生成日期范围的时候还可以加上一个时区集：

In [92]:
pd.date_range('3/9/2012 9:30',periods=10,freq='D',tz='UTC')

DatetimeIndex(['2012-03-09 09:30:00+00:00', '2012-03-10 09:30:00+00:00',
               '2012-03-11 09:30:00+00:00', '2012-03-12 09:30:00+00:00',
               '2012-03-13 09:30:00+00:00', '2012-03-14 09:30:00+00:00',
               '2012-03-15 09:30:00+00:00', '2012-03-16 09:30:00+00:00',
               '2012-03-17 09:30:00+00:00', '2012-03-18 09:30:00+00:00'],
              dtype='datetime64[ns, UTC]', freq='D')

从单纯到本地化的转换是通过tz_localize方法处理的：

In [93]:
ts_utc = ts.tz_localize('UTC')
ts_utc

2012-03-09 09:30:00+00:00   -0.167662
2012-03-10 09:30:00+00:00    1.122850
2012-03-11 09:30:00+00:00   -1.029191
2012-03-12 09:30:00+00:00   -1.002843
2012-03-13 09:30:00+00:00   -1.324037
2012-03-14 09:30:00+00:00   -0.217383
Freq: D, dtype: float64

In [106]:
ts_utc.index

DatetimeIndex(['2012-03-09 09:30:00+00:00', '2012-03-10 09:30:00+00:00',
               '2012-03-11 09:30:00+00:00', '2012-03-12 09:30:00+00:00',
               '2012-03-13 09:30:00+00:00', '2012-03-14 09:30:00+00:00'],
              dtype='datetime64[ns, UTC]', freq='D')

一旦时间序列被本地化到某个特定时区，就可以用tz_convert将其转换到别的时区了：

In [96]:
ts_utc.tz_convert('US/Eastern')

2012-03-09 04:30:00-05:00   -0.167662
2012-03-10 04:30:00-05:00    1.122850
2012-03-11 05:30:00-04:00   -1.029191
2012-03-12 05:30:00-04:00   -1.002843
2012-03-13 05:30:00-04:00   -1.324037
2012-03-14 05:30:00-04:00   -0.217383
Freq: D, dtype: float64

对于上面这种时间序列（它跨越了美国东部时区的夏令时转变期），我们可以将其本地化到EST，然后转换为UTC或柏林时间：

In [98]:
ts_eastern = ts.tz_localize('US/Eastern')
ts_eastern.tz_convert('UTC')

2012-03-09 14:30:00+00:00   -0.167662
2012-03-10 14:30:00+00:00    1.122850
2012-03-11 13:30:00+00:00   -1.029191
2012-03-12 13:30:00+00:00   -1.002843
2012-03-13 13:30:00+00:00   -1.324037
2012-03-14 13:30:00+00:00   -0.217383
Freq: D, dtype: float64

In [100]:
ts_eastern.tz_convert('Europe/Berlin')

2012-03-09 15:30:00+01:00   -0.167662
2012-03-10 15:30:00+01:00    1.122850
2012-03-11 14:30:00+01:00   -1.029191
2012-03-12 14:30:00+01:00   -1.002843
2012-03-13 14:30:00+01:00   -1.324037
2012-03-14 14:30:00+01:00   -0.217383
Freq: D, dtype: float64

tz_localize和tz_convert也是DatetimeIndex的实例方法：

In [103]:
ts.index.tz_localize('Asia/Shanghai')

DatetimeIndex(['2012-03-09 09:30:00+08:00', '2012-03-10 09:30:00+08:00',
               '2012-03-11 09:30:00+08:00', '2012-03-12 09:30:00+08:00',
               '2012-03-13 09:30:00+08:00', '2012-03-14 09:30:00+08:00'],
              dtype='datetime64[ns, Asia/Shanghai]', freq='D')

### 操作时区意识型Timestamp对象
跟时间序列和日期范围差不多，Timestamp对象也能被从单纯型（naive）本地化为时区意识型（time zone-aware），并从一个时区转换到另一个时区：

In [105]:
stamp = pd.Timestamp('2011-03-12 04:00')
stamp_utc = stamp.tz_localize('utc')
stamp_utc.tz_convert('US/Eastern')

Timestamp('2011-03-11 23:00:00-0500', tz='US/Eastern')

在创建Timestamp时，还可以传入一个时区信息：

In [108]:
stamp_moscow = pd.Timestamp('2011-03-12 04:00',tz='Europe/Moscow')
stamp_moscow

Timestamp('2011-03-12 04:00:00+0300', tz='Europe/Moscow')

时区意识型Timestamp对象在内部保存了一个UTC时间戳值（自UNIX纪元1970年1月1日算起的纳秒数）。这个UTC值在时区转换过程中是不会发生变化的：

In [109]:
stamp_utc.value

1299902400000000000L

In [112]:
stamp_utc.tz_convert('US/Eastern').value

1299902400000000000L

当使用pandas的DateOffset对象执行时间算数运算时，运算过程会自动关注是否存在夏令时转变期：

In [114]:
#夏令时转变前30分钟
from pandas.tseries.offsets import Hour
stamp = pd.Timestamp('2012-03-12 01:30',tz='US/Eastern')
stamp

Timestamp('2012-03-12 01:30:00-0400', tz='US/Eastern')

In [115]:
stamp + Hour()

Timestamp('2012-03-12 02:30:00-0400', tz='US/Eastern')

In [116]:
#夏令时转变前90分钟
stamp = pd.Timestamp('2012-11-04 00:30',tz='US/Eastern')
stamp

Timestamp('2012-11-04 00:30:00-0400', tz='US/Eastern')

In [117]:
stamp + 2*Hour()

Timestamp('2012-11-04 01:30:00-0500', tz='US/Eastern')

### 不同时区之间的运算
如果两个时间序列的时区不同，在将它们合并到一起时，最终结果就会是UTC。由于时间戳其实是以UTC存储的，所以这是一个很简单的运算，并不需要发生任何转换：

In [119]:
rng = pd.date_range('3/7/2012 9:30',periods=10,freq='B')
ts=pd.Series(np.random.randn(len(rng)),index=rng)
ts

2012-03-07 09:30:00    0.803501
2012-03-08 09:30:00   -0.746863
2012-03-09 09:30:00    0.099870
2012-03-12 09:30:00   -1.149965
2012-03-13 09:30:00   -1.825627
2012-03-14 09:30:00    0.836288
2012-03-15 09:30:00    1.910962
2012-03-16 09:30:00    1.707944
2012-03-19 09:30:00   -0.797114
2012-03-20 09:30:00   -0.556274
Freq: B, dtype: float64

In [120]:
ts1 = ts[:7].tz_localize('Europe/London')
ts2 = ts1[2:].tz_convert('Europe/Moscow')

In [123]:
result = ts1 + ts2
result.index

DatetimeIndex(['2012-03-07 09:30:00+00:00', '2012-03-08 09:30:00+00:00',
               '2012-03-09 09:30:00+00:00', '2012-03-12 09:30:00+00:00',
               '2012-03-13 09:30:00+00:00', '2012-03-14 09:30:00+00:00',
               '2012-03-15 09:30:00+00:00'],
              dtype='datetime64[ns, UTC]', freq='B')

### 时期及其算术运算
时期（period）表示的是时间区间，Period类所表示的就是这种数据类型，其构造函数需要用到一个字符串或整数，以及表10-4的频率：

In [135]:
p = pd.Period(2007,freq='A-DEC')
p

Period('2007', 'A-DEC')

这个Period对象表示的是从2007年1月1日到2007年12月31日之间的整段时间。只需对Period对象加上或减去一个整数即可达到根据其频率进行位移的效果：

In [136]:
p + 5

Period('2012', 'A-DEC')

In [137]:
p - 2

Period('2005', 'A-DEC')

In [140]:
pd.Period('2014',freq='A-DEC') - p

7L

period_range函数可用于创建规则的时期范围：

In [141]:
rng = pd.period_range('1/1/2000','6/30/2000',freq='M')
rng

PeriodIndex(['2000-01', '2000-02', '2000-03', '2000-04', '2000-05', '2000-06'], dtype='period[M]', freq='M')

PeriodIndex类保存了一组Period，它可以在任何pandas数据结构中被用作轴索引：

In [143]:
pd.Series(np.random.randn(6),index=rng)

2000-01   -1.538345
2000-02    0.835616
2000-03   -0.970014
2000-04   -0.457981
2000-05    0.339565
2000-06    1.805551
Freq: M, dtype: float64

PeriodIndex类的构造函数还允许直接使用一组字符串：

In [144]:
values = ['2001Q3','2002Q2','2003Q1']
values

['2001Q3', '2002Q2', '2003Q1']

In [145]:
index = pd.PeriodIndex(values,freq='Q-DEC')
index

PeriodIndex(['2001Q3', '2002Q2', '2003Q1'], dtype='period[Q-DEC]', freq='Q-DEC')

### 时期的频率转换
Period和PeriodIndex对象都可以通过其asfreq方法被转换成别的频率。假设有一个年度时期，希望将其转换为当年年初或年末的一个月度时期：

In [146]:
p = pd.Period('2007',freq='A-DEC')
p.asfreq('M',how='start')

Period('2007-01', 'M')